# Laboratorio 2 - IA

## Autores

Nelson Escalante

Milton Polanco

## Problema 1

In [32]:
import numpy as np

# Array con los datos a clasificar, 
# con columnas X de 1 a 4 y filas Y de 0 a 1
joint = np.array([
    [0.09, 0.16, 0.17, 0.05],   # Y = 0
    [0.15, 0.07, 0.10, 0.21]    # Y = 1
])

# Clasificador bayesiano optimo
classifier = np.argmax(joint, axis=0)

# Error bayesiano
bayes_error = np.sum(np.min(joint, axis=0))

print("Clasificion optima por X:")
for i, y in enumerate(classifier, start=1):
    print(f"X = {i} -> Y = {y}")

print("\nError del clasificador:", bayes_error)

Clasificion optima por X:
X = 1 -> Y = 1
X = 2 -> Y = 0
X = 3 -> Y = 0
X = 4 -> Y = 1

Error del clasificador: 0.31


## Problema 2

In [33]:
x_star = 0.5 * np.log(3)

Pe = 0.5 * (np.exp(-3 * x_star) + (1 - np.exp(-x_star)))

print("Umbral x* =", x_star)
print("Error Bayesiano =", Pe)

Umbral x* = 0.5493061443340549
Error Bayesiano = 0.3075499102701248


## Problema 3

In [34]:
%pip install scikit-learn pandas numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 1. Preparación de los datos
data = {
    'Outlook': ['Rainy', 'Rainy', 'Overcast', 'Sunny', 'Sunny', 'Sunny', 'Overcast', 'Rainy', 'Rainy', 'Sunny', 'Rainy', 'Overcast', 'Overcast', 'Sunny'],
    'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot', 'Mild'],
    'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'High'],
    'Windy': ['False', 'True', 'False', 'False', 'False', 'True', 'True', 'False', 'False', 'False', 'True', 'True', 'False', 'True'],
    'PlayGolf': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']
}

df = pd.DataFrame(data)

# Codificación de variables categóricas a numéricas
encoders = {}
for col in df.columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le

X = df.drop('PlayGolf', axis=1)
y = df['PlayGolf']

# 2. Casos a predecir
# Caso 1: (Rainy, Hot, High, False)
# Caso 2: (Sunny, Hot, Normal, False)
x_test = pd.DataFrame([
    ['Rainy', 'Hot', 'High', 'False'],
    ['Sunny', 'Hot', 'Normal', 'False']
], columns=['Outlook', 'Temperature', 'Humidity', 'Windy'])

for col in x_test.columns:
    x_test[col] = encoders[col].transform(x_test[col])

# 3. Entrenamiento de modelos
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X, y)

nb = CategoricalNB()
nb.fit(X, y)

# 4. Resultados de predicción
pred_knn = knn.predict(x_test)
pred_nb = nb.predict(x_test)

def print_preds(preds, model_name):
    print(f"Predicciones con {model_name}:")
    for i, p in enumerate(preds):
        label = encoders['PlayGolf'].inverse_transform([p])[0]
        print(f"  Caso {i+1}: {label}")

print_preds(pred_knn, "KNN (k=3)")
print_preds(pred_nb, "Naïve Bayes")

# 5. Evaluación de métricas (sobre el set de entrenamiento)
def calcular_metricas(model, X, y, nombre):
    y_pred = model.predict(X)
    y_proba = model.predict_proba(X)[:, 1]
    
    print(f"\nMétricas: {nombre}")
    print(f"Accuracy:  {accuracy_score(y, y_pred):.4f}")
    print(f"Precision: {precision_score(y, y_pred):.4f}")
    print(f"Recall:    {recall_score(y, y_pred):.4f}")
    print(f"F1 Score:  {f1_score(y, y_pred):.4f}")
    print(f"ROC AUC:   {roc_auc_score(y, y_proba):.4f}")

calcular_metricas(knn, X, y, "KNN")
calcular_metricas(nb, X, y, "Naïve Bayes")

Predicciones con KNN (k=3):
  Caso 1: No
  Caso 2: Yes
Predicciones con Naïve Bayes:
  Caso 1: No
  Caso 2: Yes

Métricas: KNN
Accuracy:  0.8571
Precision: 0.8889
Recall:    0.8889
F1 Score:  0.8889
ROC AUC:   0.8556

Métricas: Naïve Bayes
Accuracy:  0.9286
Precision: 0.9000
Recall:    1.0000
F1 Score:  0.9474
ROC AUC:   0.9222


### Comparación de resultados
Al evaluar ambos modelos sobre el conjunto de datos de entrenamiento:
* **Naïve Bayes:** suele presentar un desempeño más robusto en este problema clásico (Weather dataset), ya que las variables se asumen independientes y el modelo probabilístico captura bien la tendencia general a pesar del poco volumen de datos.
* **KNN:** es más sensible a la escala y a la vecindad local. Con $k=3$, el modelo tiende a memorizar el ruido si los datos son escasos, aunque ofrece resultados muy similares en precisión para este caso específico.

De acuerdo con las métricas obtenidas (Accuracy y ROC AUC), el clasificador de **Naïve Bayes** es preferible para este escenario por su estabilidad probabilística.

### ¿Cómo construir un clasificador bueno con tan pocos datos?
Para construir un clasificador eficiente cuando el dataset es pequeño (como este), se recomiendan las siguientes estrategias:
1.  **Modelos con alto sesgo (High Bias):** Utilizar algoritmos simples como Naïve Bayes o Regresión Logística. Estos modelos no requieren grandes cantidades de datos para converger y son menos propensos al overfitting en muestras pequeñas.
2.  **Validación Cruzada:** Emplear técnicas como Leave-One-Out Cross-Validation (LOOCV), donde se entrena el modelo N veces usando N-1 datos, asegurando que cada observación se use para validar.
3.  **Reducción de dimensionalidad:** Mantener solo las variables más relevantes para evitar la "maldición de la dimensionalidad".
4.  **Suavizado (Smoothing):** En el caso de Bayes, se recomienda usar suavizado de Laplace para evitar que probabilidades de cero invaliden el cálculo.